# Part 5
If your environment got too slow and unwieldy, part 5 is extracted by itself to build a new notebook with a long NDVI time series

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import rasterio
from rasterio.plot import show, reshape_as_image
from pyproj import Transformer
from satsearch import Search
import numpy.ma as ma
from rasterio import features
import geopandas 

%matplotlib inline

# Sentinel-2 STAC API
url = "https://earth-search.aws.element84.com/v0/"

# Bounding Box delineating the spatial extent for NDVI mapping
bbox = [-110.74772571639987, 32.270431012618026, -110.70996215904789, 32.29386169894274]

# The date range for mapping NDVI overtime
date_range = "2021-07-05/2021-08-02"

def image_search(bbox, date_range, scene_cloud_tolerance):
    """
    Using SatSearch find all Sentinel-2 images
    that meet our criteria
    """
    
    search = Search(
        bbox=bbox,
        datetime=date_range,
        query={
            "eo:cloud_cover": {"lt": scene_cloud_tolerance}
        },  
        collections=["sentinel-s2-l2a-cogs"],
        url=url,
    )

    return search.items()

transform_window = None
def range_request(image_url, bbox):
    """
    Request and read just the required pixels from the COG
    """
    
    with rasterio.open(image_url) as src:
        coord_transformer = Transformer.from_crs("epsg:4326", src.crs)
        # calculate pixels to be streamed from the COG
        coord_upper_left = coord_transformer.transform(bbox[3], bbox[0])
        coord_lower_right = coord_transformer.transform(bbox[1], bbox[2])
        pixel_upper_left = src.index(coord_upper_left[0], coord_upper_left[1])
        pixel_lower_right = src.index(coord_lower_right[0], coord_lower_right[1])
         
                
        # request only the bytes in the window
        window = rasterio.windows.Window.from_slices(
            (pixel_upper_left[0], pixel_lower_right[0]),
            (pixel_upper_left[1], pixel_lower_right[1]),
        )

        # The affine transform - This will allow us to 
        # translate pixels coordiantes back to geospatial coordiantes
        transform_window = rasterio.windows.transform(window,src.transform)
        
        bands = 1
        if "TCI" in image_url: # aka True Colour Image aka RGB
            bands = [1, 2, 3]

        subset = src.read(bands, window=window)
        return(subset, transform_window)



fields = geopandas.read_file('neighborhood_samples.geojson').to_crs(epsg=32612)


def mask_ndvi_by_field(image, field_geom):
    """
    returns a numpy mask
    """
    mask = features.geometry_mask(field_geom, 
                                out_shape=(image['ndvi'].shape[0],  image['ndvi'].shape[1]),
                                transform=image['transform_window'], 
                                all_touched=False, 
                                invert=False)
    ndvi_masked = ma.masked_array(image['ndvi'], mask)
    return ndvi_masked

## Part 5: NDVI Time Series 

Finally we are going to calculate the average NDVI within one of the polygons over a lengthy time period.

First we will re-search but for a longer time period. This is largely a copy-paste from earlier in this notebook but expediting the imagery for having NDVI for the full time series.

In [ ]:

# The date range for mapping NDVI overtime
date_range = "2020-01-01/2021-12-31"



long_series_images = []
items = image_search(bbox, date_range, 5)
items.summary()

rgb = items[0].asset("visual")["href"]
rgb_subset, transform_window = range_request(rgb, bbox)

for item in items:
    
    # Refs to long_series_images
    red = item.asset("red")["href"]
    nir = item.asset("nir")["href"]
    date = item.date.strftime("%m/%d/%Y")

    # Streamed pixels within bbox
    red_subset, transform_window = range_request(red, bbox)
    nir_subset, transform_window = range_request(nir, bbox)

    # Calculate NDVI
    ndvi_subset = (nir_subset.astype(float) - red_subset.astype(float)) / (
        nir_subset + red_subset
    )
    
    # Store the data for further processing
    long_series_images.append(
        {"date": date, "ndvi": ndvi_subset,'transform_window': transform_window}
    )

# reverse list as to be oldest to newest
long_series_images.reverse()

In [ ]:
print(len(long_series_images))
print(long_series_images[0].keys())

Next, subset the data by masking by each "field". In this case we are going to add a new entry to each of the `long_series_image` `dict`s containing an `ndvi_fields` as a key. This will resolve to another `dict` containing keys for all the different values of the `area` column in the `GeopandasDataFrame` that we downloaded for our "fields". The choice of `area` as an attribute name was probably a poor choice. It's a text field with values like "riparian" and "hood". Thus, each element of `image['ndvi_fields']` will have the masked NDVI data for that date and that feature.

In [ ]:
for image in long_series_images:
    ndvi_fields = {}
    
    for index, row in fields.iterrows():
        field_mask = mask_ndvi_by_field(image,row.geometry)
        ndvi_fields[row.area] = field_mask
        image['ndvi_fields'] = ndvi_fields

In [ ]:


items = image_search(bbox, date_range, 5)

rgb = items[0].asset("visual")["href"]
rgb_subset, transform_window = range_request(rgb, bbox)

In [ ]:
# Use geopandas `loc` to select the field where field_id == 6
riparian = fields.loc[fields['area'] == 'riparian']

fig, ax = plt.subplots(1, figsize=(12,12))
    
rgb_axes = show(rgb_subset,
                transform=transform_window, 
                ax=ax,
                alpha=.75,
                cmap="RdYlGn", 
                title="Field 6")
fields.boundary.plot(ax=ax, color='skyblue', linewidth=4)
riparian.boundary.plot(ax=ax, color='yellow', linewidth=6)                             
rgb_axes.ticklabel_format(style ='plain') # show full y-coords

Finally... This is where we graph the NDVI over one of our polygon features.

In [ ]:
dates = np.array([long_series_images[i]['date'] for i in range(len(long_series_images))])
ndvi_values = np.array([long_series_images[i]['ndvi_fields']["riparian"].mean() for i in range(len(long_series_images))])

fig, ax = plt.subplots(figsize=(20, 8))

ax.plot_date(dates, ndvi_values, marker='', linestyle='-')
fig.autofmt_xdate()
fig.suptitle('Mean NDVI for Riparian (2020-2021)', fontsize=24)
plt.xlabel('Date', fontsize=18)
plt.ylabel('Mean NDVI', fontsize=18)

plt.show()

## Deliverable 5 - `ndvi-time-series.png`

Save a screenshot of the time series graph above as `ndvi-time-series.png`

## Final Deliverables
Submit an open pull request from the `rasterio` branch to be merged with `master` containing these 5 files. Do not merge your pull request. 
- `stac-items.png`
- `image-numpy.png`
- `image-plots.png`
- `fields.png`
- `ndvi-time-series.png`